# Script to run LePhare on a matched table

## Requirements

This notebook is supposed to be run on a matched LePhare input file after carrying out the matching using the `match_tables.ipynb` notebook.

## Structure

- Load the region and the relevant matched and processd files
- Running `LePhare`:
  - Collect the filter information
  - Collect the template information
  - Run the photo-z-routine

In [2]:
import os
import function_package as fp
import logging
logging.getLogger().setLevel("INFO")

logging.info("Modules loaded successfully")

INFO:root:Modules loaded successfully


In [7]:

REGION = fp.Region(load_from_disk=True)
logging.info(REGION)

INFO:root:Region constrained to 125.0000 <= RA <= 130.0000 and -5.0000 <= DEC <= 0.0000.
This corresponds to a linear (!) size of 25 deg^2
